In [1]:
import cv2
import numpy as np
import os
from random import shuffle
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers,models,Sequential
from sklearn.model_selection import train_test_split
from keras.models import Sequential,Model,Input
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping,ReduceLROnPlateau
from tensorflow.keras.layers import Activation, BatchNormalization, Conv2D, Dense, Dropout, Flatten, MaxPooling2D
from tensorflow.keras.regularizers import l2
import keras
import tensorflow_hub as hub
from keras.utils.data_utils import get_file
import tensorflow.compat.v2 as tf

from keras import backend
from keras.applications import imagenet_utils
from keras.engine import training
from keras.layers import VersionAwareLayers
from keras.utils import data_utils
from keras.utils import layer_utils
import warnings
warnings.filterwarnings("ignore")

In [2]:
Train_Data='/kaggle/input/nn23-sports-image-classification/Train'
Test_Data='/kaggle/input/nn23-sports-image-classification/Test'

In [3]:
def create_label(image_name):
    
    if 'Basketball' in image_name:
      return 0
    elif 'Football' in image_name:
      return 1
    elif 'Rowing' in image_name:
      return 2
    elif 'Swimming' in image_name:
      return 3
    elif 'Tennis' in image_name:
      return 4
    elif 'Yoga' in image_name:
      return 5

In [4]:
Img_size=224

In [5]:

def create_traning_data():
  training_data=[]
  for img in os.listdir(Train_Data):

    path = os.path.join(Train_Data, img)
    img_data = cv2.imread(path,1)    
    img_data = cv2.resize(img_data, (Img_size, Img_size))    
    training_data.append([np.array(img_data), create_label(img)])
  shuffle(training_data)  
  return training_data

In [6]:
def create_testing_data():
  Testing_data=[]
  for img in os.listdir(Test_Data):
    path = os.path.join(Test_Data, img)
    img_data = cv2.imread(path,1)    
    img_data = cv2.resize(img_data, (Img_size, Img_size))    
    Testing_data.append([np.array(img_data), img])
  shuffle(Testing_data)    
  return Testing_data

In [7]:

X_train=[]
Y_train=[]
train_data=create_traning_data()
for x,y in train_data:
  X_train.append(x)
  Y_train.append(y)
X_train=np.array(X_train).reshape(-1,Img_size,Img_size,3)/255
Y_train=np.array(Y_train).reshape(-1,)
datagen=ImageDataGenerator(
  
                                  zoom_range=0.2,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  fill_mode='nearest')
datagen.fit(X_train)

x_train,x_test,y_train,y_test=train_test_split(X_train,Y_train,test_size=0.1,stratify=Y_train,random_state=10)


In [8]:

X_test=[]
images_name=[]
test_data=create_testing_data()
for x,y in test_data:
  X_test.append(x)
  images_name.append(y)
X_test=np.array(X_test).reshape(-1,Img_size,Img_size,3)/255


In [9]:
def vgg19():
    input_shape= Input(shape=(224,224,3))
    # Block 1
    x = layers.Conv2D(
        64, (3, 3), activation="relu", padding="same", name="block1_conv1"
    )(input_shape)
    x = layers.Conv2D(
        64, (3, 3), activation="relu", padding="same", name="block1_conv2"
    )(x)
    x = layers.MaxPooling2D((2, 2), strides=(2, 2), name="block1_pool")(x)

    # Block 2
    x = layers.Conv2D(
        128, (3, 3), activation="relu", padding="same", name="block2_conv1"
    )(x)
    x = layers.Conv2D(
        128, (3, 3), activation="relu", padding="same", name="block2_conv2"
    )(x)
    x = layers.MaxPooling2D((2, 2), strides=(2, 2), name="block2_pool")(x)

    # Block 3
    x = layers.Conv2D(
        256, (3, 3), activation="relu", padding="same", name="block3_conv1"
    )(x)
    x = layers.Conv2D(
        256, (3, 3), activation="relu", padding="same", name="block3_conv2"
    )(x)
    x = layers.Conv2D(
        256, (3, 3), activation="relu", padding="same", name="block3_conv3"
    )(x)
    x = layers.Conv2D(
        256, (3, 3), activation="relu", padding="same", name="block3_conv4"
    )(x)
    x = layers.MaxPooling2D((2, 2), strides=(2, 2), name="block3_pool")(x)

    # Block 4
    x = layers.Conv2D(
        512, (3, 3), activation="relu", padding="same", name="block4_conv1"
    )(x)
    x = layers.Conv2D(
        512, (3, 3), activation="relu", padding="same", name="block4_conv2"
    )(x)
    x = layers.Conv2D(
        512, (3, 3), activation="relu", padding="same", name="block4_conv3"
    )(x)
    x = layers.Conv2D(
        512, (3, 3), activation="relu", padding="same", name="block4_conv4"
    )(x)
    x = layers.MaxPooling2D((2, 2), strides=(2, 2), name="block4_pool")(x)

    # Block 5
    x = layers.Conv2D(
        512, (3, 3), activation="relu", padding="same", name="block5_conv1"
    )(x)
    x = layers.Conv2D(
        512, (3, 3), activation="relu", padding="same", name="block5_conv2"
    )(x)
    x = layers.Conv2D(
        512, (3, 3), activation="relu", padding="same", name="block5_conv3"
    )(x)
    x = layers.Conv2D(
        512, (3, 3), activation="relu", padding="same", name="block5_conv4"
    )(x)
    x = layers.MaxPooling2D((2, 2), strides=(2, 2), name="block5_pool")(x)
    

  
    vgg19=Model(input_shape,x)

    return vgg19


In [10]:
vgg19_model=vgg19()



2022-12-28 06:47:03.712088: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-28 06:47:03.834808: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-28 06:47:03.835590: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-28 06:47:03.836759: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [11]:
WEIGHTS_PATH_NO_TOP = (
    "https://storage.googleapis.com/tensorflow/"
    "keras-applications/vgg19/"
    "vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5"
)
weights_path = data_utils.get_file(
                "vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5",
                WEIGHTS_PATH_NO_TOP,
                cache_subdir="models",
                file_hash="253f8cb515780f3b799900260a226db6",
            )
vgg19_model.load_weights(weights_path)

80150528/80134624 [==============================] - 4s 0us/step


In [12]:
x = layers.Flatten(name="flatten")(vgg19_model.output)
x = layers.Dense(4096, activation="relu", name="fc1")(x)
x = layers.Dense(4096, activation="relu", name="fc2")(x)
x = layers.Dense(6, activation="softmax", name="predictions_f")(x)
model=Model(vgg19_model.input,x)


In [13]:
optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer,
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy'])

In [14]:
model.fit(x_train,y_train,epochs=30,validation_data=(x_test,y_test))

2022-12-28 06:47:32.769407: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 910393344 exceeds 10% of free system memory.
2022-12-28 06:47:33.796000: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 910393344 exceeds 10% of free system memory.
2022-12-28 06:47:34.541769: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/30


2022-12-28 06:47:36.288341: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


48/48 [==============================] - 27s 320ms/step - loss: 1.6075 - accuracy: 0.3948 - val_loss: 1.3313 - val_accuracy: 0.4675
Epoch 2/30
48/48 [==============================] - 13s 261ms/step - loss: 1.2788 - accuracy: 0.4907 - val_loss: 1.0860 - val_accuracy: 0.5858
Epoch 3/30
48/48 [==============================] - 12s 260ms/step - loss: 1.0371 - accuracy: 0.6296 - val_loss: 0.9473 - val_accuracy: 0.6450
Epoch 4/30
48/48 [==============================] - 12s 260ms/step - loss: 0.7293 - accuracy: 0.7269 - val_loss: 0.7700 - val_accuracy: 0.7337
Epoch 5/30
48/48 [==============================] - 12s 259ms/step - loss: 0.6889 - accuracy: 0.7560 - val_loss: 0.7171 - val_accuracy: 0.6923
Epoch 6/30
48/48 [==============================] - 12s 258ms/step - loss: 0.5139 - accuracy: 0.8201 - val_loss: 0.8407 - val_accuracy: 0.7101
Epoch 7/30
48/48 [==============================] - 12s 260ms/step - loss: 0.5046 - accuracy: 0.8142 - val_loss: 0.6152 - val_accuracy: 0.7870
Epoch 8/30

In [15]:
model_details=model.fit_generator(datagen.flow(x_train,y_train,batch_size=32),epochs=30,
                                 steps_per_epoch=len(X_train)/32,validation_data=(x_test,y_test),
                                 verbose=2)

Epoch 1/30
52/52 - 18s - loss: 0.8569 - accuracy: 0.7295 - val_loss: 0.6609 - val_accuracy: 0.7515


In [16]:
#pure model accuracy 86 in private leaderboard

def create_model():
    model = Sequential([
        Conv2D(filters=128, kernel_size=(5, 5), padding='valid', input_shape=(224, 224, 3)),
        Activation('relu'),
        MaxPooling2D(pool_size=(2, 2)),
        BatchNormalization(),
        
        Conv2D(filters=64, kernel_size=(3, 3), padding='valid', kernel_regularizer=l2(0.00005)),
        Activation('relu'),
        MaxPooling2D(pool_size=(2, 2)),
        BatchNormalization(),
        
        Conv2D(filters=32, kernel_size=(3, 3), padding='valid', kernel_regularizer=l2(0.00005)),
        Activation('relu'),
        MaxPooling2D(pool_size=(2, 2)),
        BatchNormalization(),
         Flatten(),
        
        Dense(units=256, activation='relu'),
        Dropout(0.5),
        Dense(units=6, activation='softmax')
    ])
    
    return model
    

In [17]:
cnn_model = create_model()

In [18]:
cnn_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 220, 220, 128)     9728      
_________________________________________________________________
activation (Activation)      (None, 220, 220, 128)     0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 110, 110, 128)     0         
_________________________________________________________________
batch_normalization (BatchNo (None, 110, 110, 128)     512       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 108, 108, 64)      73792     
_________________________________________________________________
activation_1 (Activation)    (None, 108, 108, 64)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 54, 54, 64)        0

In [19]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=np.sqrt(0.1), patience=5)

In [20]:
optimizer = Adam(learning_rate=0.0001)

In [21]:
cnn_model.compile(optimizer=optimizer,
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy'])

In [23]:
history = cnn_model.fit(x_train,y_train,epochs=30,
                       callbacks=[reduce_lr],validation_data=(x_test,y_test))

2022-12-28 06:56:06.708329: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 910393344 exceeds 10% of free system memory.


Epoch 1/30
48/48 [==============================] - 3s 68ms/step - loss: 0.1494 - accuracy: 0.9557 - val_loss: 2.3170 - val_accuracy: 0.3195
Epoch 2/30
48/48 [==============================] - 3s 63ms/step - loss: 0.0913 - accuracy: 0.9716 - val_loss: 3.1331 - val_accuracy: 0.3373
Epoch 3/30
48/48 [==============================] - 3s 62ms/step - loss: 0.0744 - accuracy: 0.9788 - val_loss: 2.4276 - val_accuracy: 0.3728
Epoch 4/30
48/48 [==============================] - 3s 66ms/step - loss: 0.0726 - accuracy: 0.9788 - val_loss: 3.0311 - val_accuracy: 0.3609
Epoch 5/30
48/48 [==============================] - 3s 65ms/step - loss: 0.0515 - accuracy: 0.9868 - val_loss: 2.1850 - val_accuracy: 0.4497
Epoch 6/30
48/48 [==============================] - 3s 65ms/step - loss: 0.0388 - accuracy: 0.9921 - val_loss: 1.3572 - val_accuracy: 0.5976
Epoch 7/30
48/48 [==============================] - 3s 63ms/step - loss: 0.0379 - accuracy: 0.9914 - val_loss: 1.0024 - val_accuracy: 0.7041
Epoch 8/30
48

In [23]:
model_details=cnn_model.fit_generator(datagen.flow(x_train,y_train,batch_size=32),epochs=30,
                                 steps_per_epoch=len(X_train)/32,validation_data=(x_test,y_test),
                                 verbose=2)

Epoch 1/30
52/52 - 16s - loss: 1.1095 - accuracy: 0.7361 - val_loss: 0.8741 - val_accuracy: 0.7811


In [15]:
cnn_model.evaluate(x_test,y_test)

2022-12-28 02:13:15.737574: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-12-28 02:13:16.876064: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


6/6 [==============================] - 7s 47ms/step - loss: 0.0072 - accuracy: 1.0000


[0.007241294253617525, 1.0]

In [18]:
cnn_model.save('cnn_model3.h5')

In [14]:
from tensorflow.keras.models import load_model

cnn_model = tf.keras.models.load_model('/kaggle/input/trysc-h11/cnn_model3.h5')

In [24]:
pridctions=cnn_model.predict(X_test)

In [26]:
pridctions=np.argmax(pridctions,axis=1)

In [27]:
pridctions

array([1, 4, 1, 1, 0, 5, 2, 5, 1, 1, 1, 5, 2, 4, 5, 2, 2, 3, 5, 5, 2, 0,
       0, 3, 4, 2, 4, 0, 3, 0, 3, 3, 0, 3, 5, 5, 4, 3, 3, 5, 1, 2, 4, 5,
       0, 3, 5, 0, 4, 5, 5, 3, 5, 2, 3, 5, 4, 1, 3, 1, 4, 5, 1, 0, 2, 5,
       4, 1, 1, 2, 3, 0, 5, 5, 5, 4, 4, 2, 1, 1, 5, 5, 5, 3, 1, 3, 1, 0,
       1, 1, 5, 3, 0, 3, 5, 1, 3, 4, 5, 0, 3, 3, 1, 1, 1, 5, 2, 1, 5, 0,
       3, 4, 3, 3, 0, 4, 4, 1, 4, 2, 5, 1, 1, 2, 2, 1, 2, 2, 3, 0, 5, 5,
       4, 0, 1, 1, 1, 2, 1, 3, 3, 1, 3, 1, 1, 0, 1, 2, 0, 2, 1, 5, 1, 2,
       1, 3, 3, 4, 1, 4, 0, 2, 3, 2, 2, 3, 5, 5, 0, 3, 5, 0, 5, 4, 1, 1,
       5, 0, 0, 3, 1, 4, 5, 5, 2, 2, 5, 5, 5, 1, 5, 1, 3, 5, 4, 1, 3, 1,
       5, 4, 3, 3, 0, 5, 0, 1, 0, 3, 1, 1, 4, 2, 5, 0, 5, 1, 0, 1, 4, 4,
       0, 3, 1, 0, 1, 2, 1, 2, 1, 5, 1, 5, 3, 1, 0, 5, 5, 2, 5, 0, 5, 0,
       3, 5, 1, 5, 2, 1, 1, 4, 3, 4, 4, 1, 2, 2, 0, 1, 1, 5, 2, 4, 2, 1,
       4, 1, 1, 1, 3, 5, 4, 1, 0, 5, 5, 3, 1, 3, 2, 0, 2, 2, 2, 3, 1, 2,
       3, 3, 0, 1, 0, 1, 5, 4, 4, 1, 2, 1, 1, 2, 0,

In [29]:
import pandas as pd
rawData = {"image_name":images_name , "label":pridctions}

Data = pd.DataFrame(rawData)

Data = Data.sort_values("image_name")


Data.to_csv("خير.csv",index=False)

In [50]:
cnn_model.save_weights("best_weights")


In [47]:
display(Data)

,image_name,label
53,0.jpg,4
191,1.jpg,4
345,10.jpg,5
208,100.jpg,5
154,101.jpg,1
...,...,...
513,95.jpg,2
270,96.jpg,4
587,97.jpg,2
361,98.jpg,0


In [22]:
model.evaluate(X_test,Y_test)

11/11 [==============================] - 3s 295ms/step - loss: 0.6741 - accuracy: 0.8012


[0.6740801930427551, 0.8011869192123413]

In [ ]:
plot_mo

In [19]:
Model.evaluate(X_valid,Y_valid)

23/23 [==============================] - 0s 21ms/step - loss: 2.2486 - accuracy: 0.4288


[2.2485852241516113, 0.4287690222263336]